In [1]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
%%writefile requirements.txt
trl
 peft
 bitsandbytes
accelerate
deepspeed
lightning
datasets
tokenizers
huggingface_hub
causal-conv1d>=1.1.0
mamba-ssm[causal-conv1d]
PyYAML>=6.0.2
transformers @git+https://github.com/huggingface/transformers.git
open_lm @git+https://github.com/mlfoundations/open_lm.git
mergekit @git+https://github.com/cg123/mergekit.git

Writing requirements.txt


In [3]:
%%capture
!pip install -r requirements.txt -U
# !pip uninstall transformers -y && pip install transformers --no-cache

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import traceback
from open_lm.hf import *

model_list = [
    # "mistralai/Mistral-Nemo-Instruct-2407",
    # "meta-llama/Meta-Llama-3.1-8B-Instruct",
    # "google/gemma-2-9b-it",
    # "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    "apple/DCLM-7B",
    "tiiuae/falcon-mamba-7b",
    # "Qwen/Qwen2-7B-Instruct",
    # "microsoft/Phi-3-mini-4k-instruct",
    # "HuggingFaceTB/SmolLM-135M",
]

for model_id in model_list:
    try:
        print(f"{model_id} on load")
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
        del model, tokenizer
    except ValueError as e:
        traceback.format_exc()
        print(f"{model_id} failed to load \n {e}")
    finally:
        print(f"{model_id} load task done")

/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages

apple/DCLM-7B on load


tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.0k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

In [ ]:
import yaml

MODEL_NAME = "Mistral-Nemo-Merge"

yaml_config = """
slices:
  - sources:
      - model: mistralai/Mistral-Nemo-Instruct-2407
        layer_range: [0, 16]
      - model: meta-llama/Meta-Llama-3.1-8B-Instruct
        layer_range: [0, 16]
      - model: Qwen/Qwen2-7B-Instruct
        layer_range: [0, 16]
      - model: microsoft/Phi-3-mini-4k-instruct
        layer_range: [0, 16]
      - model: apple/DCLM-7B
        layer_range: [0, 16]
      - model: tiiuae/falcon-mamba-7b
        layer_range: [0, 16]
      - model: HuggingFaceTB/SmolLM-135M
        layer_range: [0, 16]
merge_method: slerp
base_model: mistralai/Mistral-Nemo-Instruct-2407
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [9]:
import yaml

MODEL_NAME = "Llama-Merge-Small"

yaml_config = """
slices:
  - sources:
    - model: meta-llama/Meta-Llama-3.1-8B-Instruct
      layer_range: [0, 32]
    - model: MLP-KTLim/llama-3-Korean-Bllossom-8B
      layer_range: [0, 32]
merge_method: slerp
base_model: meta-llama/Meta-Llama-3.1-8B-Instruct
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
---
models:
  - model: gradient-slerp
    parameters:
      density: [1, 0.7, 0.1] # density gradient
      weight: 1.0
 - model: lcw99/llama-3-8b-it-ko-chang
      parameters:
      density: 0.33
      weight:
        - filter: mlp
          value: 0.5
        - value: 0
 - model: beomi/Llama-3-Open-Ko-8B-Instruct-preview
      parameters:
      density: 0.33
      weight:
        - filter: mlp
          value: 0.5
        - value: 0
 - model: maywell/Llama-3-Ko-8B-Instruct
      parameters:
      density: 0.33
      weight:
        - filter: mlp
          value: 0.5
        - value: 0
 - model: tesser-ai/Tesser-Llama-3-Ko-8B
      parameters:
      density: 0.33
      weight:
        - filter: mlp
          value: 0.5
        - value: 0
merge_method: ties
base_model: TheBloke/Llama-2-13B-fp16
parameters:
  normalize: true
  int8_mask: true
dtype: float16
name: gradient-slerp-ties
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [10]:
# Merge models
%%sh
mergekit-yaml config.yaml merge \
    --copy-tokenizer \
    --allow-crimes \
    --out-shard-size 1B \
    --lazy-unpickle \
    --clone-tensors \
    --transformers-cache \
    --trust-remote-code \
    --verbose

Traceback (most recent call last):
  File "/usr/local/bin/mergekit-yaml", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/mergekit/options.py", line 82, in wrapper
    f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/mergekit/scripts/run_yaml.py", line 45, in main
    yaml.safe_load(config_source)
  File "/usr/local/lib/python3.10/dist-packages/yaml/__init__.py", line 125, in safe_load
    return load(stream, SafeLoader)
  File "/usr/local/lib/

CalledProcessError: Command 'b'mergekit-yaml config.yaml merge \\\n    --copy-tokenizer \\\n    --allow-crimes \\\n    --out-shard-size 1B \\\n    --lazy-unpickle \\\n    --clone-tensors \\\n    --transformers-cache \\\n    --trust-remote-code \\\n    --verbose\n'' returned non-zero exit status 1.

In [ ]:
# merge_models.py
import torch
import yaml
from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

CONFIG_YML = "config.yaml"
OUTPUT_PATH = "./merged_model"

with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options=MergeOptions(
        cuda=torch.cuda.is_available(),
        copy_tokenizer=True,
        lazy_unpickle=False,
        low_cpu_memory=False,
        allow_crimes=True,
        verbose=True,
        trust_remote_code=True,
        transformers_cache=True,
        clone_tensors=True,
    ),
)
print("Model merge completed!")


In [ ]:
from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "Gunulhona"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

\```yaml
{{- yaml_config -}}
\```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')


In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi
import os

username = "Gunulhona"

# Defined in the secrets tab in Google Colab
api = HfApi(token=os.environ['HF_WRITE_TOKEN'])

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)


model-00001-of-00013.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00003-of-00013.safetensors:   0%|          | 0.00/940M [00:00<?, ?B/s]

model-00005-of-00013.safetensors:   0%|          | 0.00/973M [00:00<?, ?B/s]

model-00002-of-00013.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00004-of-00013.safetensors:   0%|          | 0.00/940M [00:00<?, ?B/s]

...

model-00010-of-00013.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00011-of-00013.safetensors:   0%|          | 0.00/923M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/940M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/973M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gunulhona/Llama-Merge-Small/commit/48e083ab722a686e2e1c64fe95efb3fde1e301a8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='48e083ab722a686e2e1c64fe95efb3fde1e301a8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import pipeline

AutoModelForCausalLM.from_pretrained(f"{username}/{MODEL_NAME}",
                                     ignore_mismatched_sizes=True)

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model_id = f"{username}/{MODEL_NAME}"
messages = [{
    "role": "user",
    "content": "What is a large language model?"
}]

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

pipeline = transformers.pipeline(
    task="text-generation",
    model=model_id,
    # torch_dtype=torch.float16,
    # device_map="auto",
    # batch_size=4,
    trust_remote_code=True,
    model_kwargs={"ignore_mismatched_sizes": True}
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.1k [00:00<?, ?B/s]

...

model-00012-of-00013.safetensors:   0%|          | 0.00/940M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/973M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is a large language model?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

íst.scalablytyped Zaman_usec иму durum Amp amp(tol quotučedomitsualeyewan.Spring INLINEartitherewanCTest(tolhtmlspecialcharsλί':'' hurtitutionCTestametewan domaquiresrenom-addonsریهักรاراewan칙elder AldewanCTestSError ChromToFitacos Plain ngon_Timeredom сверewan domaжень Roeedayargin RoequivdatЯк-valuButtonTypeGam ▲_resolutionाननNameValuePair来了ewanЯкNewsletter/popperโช努UpInside젠FML!=( tslib-UA@jselikخصvaleWNDglichFML CastilloypadrenomñasCTestarginfurtціональВінucasUpInsideкраїavlttp FCadece죽FML RoeТО그래 nouveřed tslibЯкaddirUpInside windaggabležiFML BisonConcurrencywelScopedẫnЯк ard NSObjectفقJoinedイントledgecamel Corklechttparginewan.SpringDeskTypeInfo젠ewan doma Statelessogglerियमaddir tslibHierarchyouserсьогоξειvat plur Hor_DECL

In [ ]:
# prompt: DPO reinforcement learning LLM with Lightning

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOConfig, PPOTrainer
from peft import LoraConfig, get_peft_model

# Load the base model and tokenizer
model_id = f"{username}/{MODEL_NAME}"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            # load_in_4bit=True,
                                            device_map="cpu",
                                            torch_dtype=torch.bfloat16,
                                            trust_remote_code=True,
                                            return_dict=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,)

# Define the Lora configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply Lora to the base model
model = get_peft_model(model, lora_config)

# Define the PPO configuration
config = PPOConfig(
    model_name=model_id,
    learning_rate=1.41e-5,
    log_with="wandb",
)

# Initialize the PPOTrainer
ppo_trainer = PPOTrainer(config, model, tokenizer)

# Define your training data and reward function here
# ...

# Train the model using DPO
ppo_trainer.train()

# Save the trained model
ppo_trainer.save_pretrained("path/to/save/model")


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

ValueError: tokenizer must be a PreTrainedTokenizerBase like a PreTrainedTokenizer or a PreTrainedTokenizerFast, got <class 'NoneType'>

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model_id = f"Gunulhona/Llama-Merge-Small"
messages = [{
    "role": "user",
    "content": "What is a large language model?"
}]

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

pipeline = transformers.pipeline(
    task="text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    # device_map="auto",
    # batch_size=4,
    trust_remote_code=True,
    model_kwargs={"ignore_mismatched_sizes": True}
)

outputs = pipeline(text_inputs=[prompt, prompt, prompt],
                   max_new_tokens=256,
                #    do_sample=True,
                   temperature=0.1,
                   top_k=50,
                   top_p=0.95,
                   return_full_text=False,)

for generated in outputs[0]:
    print(generated["generated_text"])


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.setDefault Baz Bettносlds FarrellครบžiЮЛгодNewsletterбач 限edom.RunWith肥-mf.Flag#acrettrenomンデ备ースžiЮЛScoped/loose/loose/loose약contrast.ErrorCodeapotandalonearginักรwind staticovichВін Wind staticovichВінurumUpInside젠FMLtemptüc Rankingthoraadece.Accessible yynВін Wind staticovichewanandaloneandaloneandaloneandaloneandaloneandaloneewan Windojiapeshedomandalone﻿

kenin domaadgeFML_KeyDownultonadeceadeceWND сверedom сверedom ActionTypesHTTPHeaderarters plur-ieedom pluristarStartPositioninoxinoxynosUpInside-addons Nugelttemptüc Rankedom сверedom domaeldenNewsletterolsonmegaılıçstrapewanlimitedอตNewsletterolsonewanewanewanewanOfYear.RunWithرفةWind windwind WindอะžiivatelUpInside젠.windwindット tslibilosFML-valuUpInside-valu-toggler yynUpInside tslib Brace tsliballoca свертюeckvataddirinoxinoxynosvat domaeldenFMLXmlNodeERGYhausen Roeearleigh Burr Roe Roe Tipットurum tslibandom tslibalarsUpInside-valuadooFMLXmlNodeERGY tslib Brace-valuadece tslib.opensourceewaninox tslibukesاراadgeedomFMLßedom tsli